In [2]:
from PIL import Image
import numpy as np
c0 = Image.open('data/test/0/1.png')
display(c0)
c0_numpy = np.array(c0)
c0_numpy.shape

(28, 28, 4)

In [4]:
c0_numpy[:,:,:-1].shape

(28, 28, 3)

# 1. CNN

In [5]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from PIL import Image
np.random.seed(3)

In [6]:
circle = Image.open('data/handwriting/handwriting_shape/train/circle/circle001.png')
np.array(circle).shape
# 모든 이미지의 shape이 같을 필요는 없음

(24, 24, 4)

In [9]:
# 1. 데이터 생성하기 학습훈련셋 : data/handwriting/handwriting_shape/train/*
#                      테스트셋 : data/handwriting/handwriting_shape/test/*
train_datagen = ImageDataGenerator(rescale=1/255.)
train_data = train_datagen.flow_from_directory(
    'data/handwriting/handwriting_shape/train/', # train 아래 폴더명이 라벨
    target_size=(24,24), # 넘파이 배열의 이미지 크기 조정
    class_mode='categorical', # 원핫인코딩 형식으로 반환
    batch_size=3,  # 한번에 반환할 이미지와 라벨 수
)
test_datagen = ImageDataGenerator(rescale=1/255.)
test_data = test_datagen.flow_from_directory(
    'data/handwriting/handwriting_shape/test/',
    target_size=(24,24),
    class_mode='categorical',
    batch_size=3,
    shuffle=False # 이미지와 라벨 순서가 고정
)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [10]:
test_data

In [12]:
X_images, Y_labels = next(test_data) # 한 배치사이즈만큼만 데이터를 가져옴
X_images, Y_labels

(array([[[[1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ],
          ...,
          [1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ]],
 
         [[1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ],
          ...,
          [1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ]],
 
         [[1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ],
          ...,
          [1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ]],
 
         ...,
 
         [[1.       , 1.       , 1.       ],
          [1.       , 1.       , 1.       ],
          [1.       , 1.      

In [14]:
print('데이터 전체수 :', test_data.n)
print('batch_size :', test_data.batch_size)

데이터 전체수 : 15
batch_size : 3


In [17]:
# 전체가져오기
all_images = [] # 독립변수
all_labels = [] # 종속변수

for images, labels in test_data:
    all_images.append(images)
    all_labels.append(labels)
    if len(all_images) * test_data.batch_size >= test_data.n:
        break

X_test_images = np.concatenate(all_images)
Y_test_labels = np.concatenate(all_labels)

X_test_images.shape, Y_test_labels.shape

((15, 24, 24, 3), (15, 3))

In [36]:
# 2. 모델 구성
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 22, 22, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 20, 20, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 6400)              0         
                                                                 
 dense_6 (Dense)             (None, 128)               819328    
                                                                 
 dense_7 (Dense)             (None, 3)                 387       
                                                      

In [37]:
# 3. 학습설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 학습시키기
model.fit(train_data, epochs=50, validation_data=test_data)

Epoch 1/50
15/15 [==============================] - 1s 24ms/step - loss: 1.4052 - accuracy: 0.2667 - val_loss: 0.8604 - val_accuracy: 0.8667
Epoch 2/50
15/15 [==============================] - 0s 14ms/step - loss: 0.4221 - accuracy: 0.9333 - val_loss: 0.1205 - val_accuracy: 1.0000
Epoch 3/50
15/15 [==============================] - 0s 15ms/step - loss: 0.0287 - accuracy: 1.0000 - val_loss: 0.0340 - val_accuracy: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 8ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0180 - val_accuracy: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 7ms/step - loss: 7.9568e-04 - accuracy: 1.0000 - val_loss: 0.0133 - val_accuracy: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 7ms/step - loss: 5.1945e-04 - accuracy: 1.0000 - val_loss: 0.0148 - val_accuracy: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 7ms/step - loss: 3.7141e-04 - accuracy: 1.0000 - val_loss: 0.0127 - val_accuracy: 1.0000
Epoch

In [38]:
# 모델평가하기
loss, accuracy = model.evaluate(test_data)
print('loss :', loss)
print('정확도 :', accuracy*100,'%')

5/5 [==============================] - 0s 4ms/step - loss: 0.0089 - accuracy: 1.0000
loss : 0.0089033804833889
정확도 : 100.0 %


In [48]:
# 예측하기
y_hat = model.predict(test_data)
y_hat.argmax(axis=1)

5/5 [==============================] - 0s 4ms/step


array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2], dtype=int64)

In [49]:
np.set_printoptions(formatter={'float':lambda x: '{:.2f}'.format(x)})
y_hat

array([[1.00, 0.00, 0.00],
       [1.00, 0.00, 0.00],
       [1.00, 0.00, 0.00],
       [1.00, 0.00, 0.00],
       [1.00, 0.00, 0.00],
       [0.00, 1.00, 0.00],
       [0.00, 1.00, 0.00],
       [0.00, 1.00, 0.00],
       [0.00, 1.00, 0.00],
       [0.07, 0.88, 0.05],
       [0.00, 0.00, 1.00],
       [0.00, 0.00, 1.00],
       [0.00, 0.01, 0.99],
       [0.00, 0.00, 1.00],
       [0.00, 0.00, 1.00]], dtype=float32)

In [45]:
# 실제데이터
test_data.labels

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2])

# 2. 데이터 증강을 통한 CNN accuracy 올리기
- 데이터 추가 확보
- 이미지 데이터를 증강(데이터 부풀리기) : train set에서만
- 레이어층(Conv2D, MaxPooling, 배치정규화, Dense)
- 활성화함수, 옵티마이저, Dropoyt(과적합 다소 해결)

In [55]:
# 1. 데이터 생성
train_datagen = ImageDataGenerator(rescale=1/255.,
                                   rotation_range=10, # 이미지를 최대 +10~-10도 회전
                                   width_shift_range=0.2, # 가로 방향으로 최대 20% 이동
                                   height_shift_range=0.2, # 세로 방향으로 최대 20% 이동
                                   shear_range=0.7, # 최대 0.7라디안(40도정도)으로 이미지 밀림
                                   zoom_range=[0.9,1.5], # 90~150%크기로 확대/축소
                                   fill_mode='nearest', # 이미지 확대시 채우는 방식
                                   horizontal_flip=True, # 좌우반전
                                   vertical_flip=True, # 상하반전
                                  )
train_data = train_datagen.flow_from_directory(
    'data/handwriting/handwriting_shape/train/',
    target_size=(24,24),
    class_mode='categorical',
    batch_size=3
)

test_datagen = ImageDataGenerator(rescale=1/255.)
test_data = test_datagen.flow_from_directory(
    'data/handwriting/handwriting_shape/test/',
    target_size=(24,24),
    class_mode='categorical',
    batch_size=3
)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [56]:
# 2. 모델 구성
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.2))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 22, 22, 64)        1792      
                                                                 
 conv2d_13 (Conv2D)          (None, 20, 20, 64)        36928     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 10, 10, 64)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 8, 8, 64)          36928     
                                                                 
 conv2d_15 (Conv2D)          (None, 6, 6, 64)          36928     
                                                      

In [60]:
# 3. 학습설정
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 학습시키기
model.fit(train_data, epochs=50, validation_data=test_data)

Epoch 1/50
15/15 [==============================] - 1s 26ms/step - loss: 0.0307 - accuracy: 0.9778 - val_loss: 1.7563e-06 - val_accuracy: 1.0000
Epoch 2/50
15/15 [==============================] - 0s 8ms/step - loss: 0.2124 - accuracy: 0.9778 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 3/50
15/15 [==============================] - 0s 8ms/step - loss: 0.2382 - accuracy: 0.8889 - val_loss: 2.2146e-05 - val_accuracy: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 8ms/step - loss: 0.0390 - accuracy: 0.9778 - val_loss: 1.5724e-04 - val_accuracy: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 8ms/step - loss: 0.1654 - accuracy: 0.9556 - val_loss: 0.0163 - val_accuracy: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 8ms/step - loss: 0.4785 - accuracy: 0.8667 - val_loss: 4.4024e-04 - val_accuracy: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 8ms/step - loss: 0.0303 - accuracy: 0.9778 - val_loss: 2.2286e-04 - val_accuracy: 1.0000

In [61]:
# 평가하기
score = model.evaluate(test_data)
print('정확도 :', score[1]*100, '%')

5/5 [==============================] - 0s 6ms/step - loss: 1.3510e-07 - accuracy: 1.0000
정확도 : 100.0 %


In [62]:
# 예측하기 
pred = model.predict(test_data)
print(pred)

5/5 [==============================] - 0s 3ms/step
[[1.00 0.00 0.00]
 [0.00 0.00 1.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]
 [0.00 1.00 0.00]
 [1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [1.00 0.00 0.00]
 [0.00 0.00 1.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]
 [1.00 0.00 0.00]
 [1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]


In [63]:
pred.argmax(axis=1)

array([0, 2, 1, 2, 1, 0, 1, 0, 2, 1, 2, 0, 0, 1, 2], dtype=int64)